### **Practise I**

**Logging Biometric Data (CREATE)**

We will create a function that accepts data from a form about a wearable sensor's activity and writes it to the database.

**Preparation:**

- We have a configuration, db object, table model, and form ready:

```python
# config/configuration.py
class DBConfig:
    """
    Set Flask config variables
    """
    # General Config
    DEBUG = True
    SECRET_KEY = 'my_secret_something' 
    
    # Database Config (set up DB connection) - for SQLite
    SQLALCHEMY_DATABASE_URI = 'sqlite:///my_database.db' # path to the SQLite "instance" folder with ".db" file
    SQLALCHEMY_TRACK_MODIFICATIONS = False # disable warnings in terminal for session changes

# database.py
from flask_sqlalchemy import SQLAlchemy

db = SQLAlchemy()

# models.py
from database import db
import datetime 

class HeartRate(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    timestamp = db.Column(db.DateTime, default=datetime.datetime.now)
    bpm = db.Column(db.Integer, nullable=False)

# forms.py
from flask_wtf import FlaskForm
from wtforms import IntegerField, SubmitField
from wtforms.validators import DataRequired

class HeartRateForm(FlaskForm):
    bpm = IntegerField('Current heart rate (BPM)', validators=[DataRequired()])
    submit = SubmitField('Save')
```

**Assignment:**

We will use the prepared data to build the application. In the application, we will create a **view function** for the `/log_heart_rate` route (i.e., `GET` and `POST` methods), which:
1.  Creates an instance of the `HeartRateForm`.
2.  On a `GET` request, displays the `log_hr.html` template with the form = we need to create this HTML file!
3.  On a `POST` request (`validate_on_submit`):
    * Creates a new instance of the `HeartRate` model with data from the form.
    * Adds the new object to `db.session` and performs a `commit`.
    * Returns a simple message, e.g., `f"A new heart rate of {heart_rate_value} BPM has been logged."`.

In [ ]:
# config/configuration.py

class DBConfig:
    """
    Set Flask config variables
    """
    # General Config
    DEBUG = True
    SECRET_KEY = 'my_secret_something' 
    
    # Database Config (set up DB connection) - for SQLite
    SQLALCHEMY_DATABASE_URI = 'sqlite:///my_database.db' # path to the SQLite "instance" folder with ".db" file
    SQLALCHEMY_TRACK_MODIFICATIONS = False # disable warnings in terminal for session changes

In [ ]:
# database.py

from flask_sqlalchemy import SQLAlchemy

db = SQLAlchemy()

In [ ]:
# models.py

from database import db

class HeartRate(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    timestamp = db.Column(db.DateTime, nullable=False)
    bpm = db.Column(db.Integer, nullable=False)

In [ ]:
# forms.py

from flask_wtf import FlaskForm
from wtforms import IntegerField, SubmitField, validators


class HeartRateForm(FlaskForm):
    bpm = IntegerField('Current heart rate (BPM)', validators=[validators.DataRequired()])
    submit = SubmitField('Save')

In [ ]:
from config.configuration import DBConfig
from flask import Flask, render_template
from database import db  # import 'db' object
from models import HeartRate # import tab model
from forms import HeartRateForm
import datetime


app = Flask(__name__)

app.config.from_object(DBConfig) 

db.init_app(app) 


with app.app_context(): 
    db.create_all() 


@app.route('/log_heart_rate', methods=['GET', 'POST'])
def log_heart_rate():
    form = HeartRateForm()
    if form.validate_on_submit():
        heart_rate_value = form.bpm.data # data from the form

        new_reading = HeartRate(timestamp=datetime.datetime.now(), bpm=heart_rate_value)
        db.session.add(new_reading) 
        db.session.commit()
        
        return f"A new heart rate of {heart_rate_value} BPM has been logged."
    
    return render_template('log_hr.html', form=form)


if __name__ == "__main__":
    app.run()

In [ ]:
<!-- templates/log_hr.html -->

<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Log Heart Rate</title>
</head> 

<body>
    <h1>Enter new heart rate</h1>
    <form method="POST" action="">
        {{ form.hidden_tag() }} <p>  <!-- CSRF token -->
            
            {{ form.bpm.label }}<br>
            {{ form.bpm(size=20) }}
        </p>
        <p>{{ form.submit() }}</p>
    </form>
</body>

</html>

### **Practise II**

**Searching the Component Catalog (READ)**

We have a database of spare parts and components for robots. The goal is to create a route that displays only components of a given type.

**Preparation:**

- We have a configuration, db object, and table model ready:

```python
# config/configuration.py
class DBConfig:
    """
    Set Flask config variables
    """
    # General Config
    DEBUG = True
    SECRET_KEY = 'my_secret_something' 
    
    # Database Config (set up DB connection) - for SQLite
    SQLALCHEMY_DATABASE_URI = 'sqlite:///my_database.db' # path to the SQLite "instance" folder with ".db" file
    SQLALCHEMY_TRACK_MODIFICATIONS = False # disable warnings in terminal for session changes

# database.py
from flask_sqlalchemy import SQLAlchemy

db = SQLAlchemy()

# models.py
class RoboticPart(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    part_name = db.Column(db.String(100), nullable=False)
    manufacturer = db.Column(db.String(50))
    part_type = db.Column(db.String(20)) # e.g. 'sensor', 'processor' ..
```

**Populating the database (in the main application file):**

To have something to search for, we will add this code to our main file. It will create several sample components, but only if the database is empty, to prevent duplicates on every restart.

```python

with app.app_context(): # creates a table in DB and fills it with initial data
    db.create_all()
    if not RoboticPart.query.first(): # check if the table is empty
        part1 = RoboticPart(part_name='Optic sensor T-100', manufacturer='CyCorp', part_type='sensor')
        part2 = RoboticPart(part_name='MicroEngine A-3', manufacturer='MCCa', part_type='engine')
        part3 = RoboticPart(part_name='Logic chip v4', manufacturer='LaosLC', part_type='processor')
        part4 = RoboticPart(part_name='LIDAR', manufacturer='SEEThr', part_type='sensor')  
        db.session.add_all([part1, part2, part3, part4])
        db.session.commit()

```

**Assignment:**

We will use the prepared data to build the application. In the application, we will create a **view function** for the dynamic route `/parts/type/<string:type_name>`, which:
1.  Accepts the component type from the URL (`type_name`).
2.  Performs a query to the database and filters **all** records that have the given type.
3.  Passes the found components (a list) to the `show_parts.html` template.
4.  We will create an HTML file `show_parts.html` where we will display the names of all found components using a loop.

In [ ]:
# config/confugiration.py

class DBConfig:
    """
    Set Flask config variables
    """
    # General Config
    DEBUG = True
    SECRET_KEY = 'my_secret_something' 
    
    # Database Config (set up DB connection) - for SQLite
    SQLALCHEMY_DATABASE_URI = 'sqlite:///my_database.db' # path to the SQLite "instance" folder with ".db" file
    SQLALCHEMY_TRACK_MODIFICATIONS = False # disable warnings in terminal for session changes

In [ ]:
# database.py

from flask_sqlalchemy import SQLAlchemy

db = SQLAlchemy()

In [ ]:
# models.py

class RoboticPart(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    part_name = db.Column(db.String(100), nullable=False)
    manufacturer = db.Column(db.String(50))
    part_type = db.Column(db.String(20))

In [ ]:
from config.configuration import DBConfig
from flask import Flask, render_template
from database import db  
from models import RoboticPart 


app = Flask(__name__)

app.config.from_object(DBConfig) 

db.init_app(app) 


with app.app_context(): # creates a table in DB and fills it with initial data
    db.create_all()
    if not RoboticPart.query.first(): # check if the table is empty
        part1 = RoboticPart(part_name='Optic sensor T-100', manufacturer='CyCorp', part_type='sensor')
        part2 = RoboticPart(part_name='MicroEngine A-3', manufacturer='MCCa', part_type='engine')
        part3 = RoboticPart(part_name='Logic chip v4', manufacturer='LaosLC', part_type='processor')
        part4 = RoboticPart(part_name='LIDAR', manufacturer='SEEThr', part_type='sensor')  
        db.session.add_all([part1, part2, part3, part4])
        db.session.commit()


@app.route('/parts/type/<string:type_name>')
def show_parts_by_type(type_name):
    found_parts = RoboticPart.query.filter_by(part_type=type_name).all()
    return render_template('show_parts.html', parts=found_parts, part_type=type_name)


if __name__ == "__main__":
    app.run()

In [ ]:
<!-- templates/show_parts.html -->

<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>On Stock</title>
</head>

<body>
    <h1>Found components of type: {{ part_type }}</h1>

    {% if parts %}
        <ul>
        {% for part in parts %}
            <li>{{ part.part_name }} (Made by: {{ part.manufacturer }})</li>
        {% endfor %}
        </ul>
    {% else %}
        <p>No component found.</p>
    {% endif %}
</body>

</html>

### **Project (Homework): Upgrade to the Central Crew Databank**

**Mission:** Our improvised records of the crew and new applicants, stored in the program's volatile memory, are no longer sufficient for a long-term mission. It's time to upgrade our systems to a **persistent databank** using SQLAlchemy to ensure that crew data survives system restarts and potential solar flares.


**Basic application structure:**
- my_project/
  - my_app.py
  - routes.py
  - forms.py
  - **database.py**
  - **models.py**
  - templates/
  - static/
  - config/


**1. Establishing the Databank and Defining Schemas:**
- Create the files `database.py` and `models.py`.
- In `models.py`, define the `CrewMember` database model, which will correspond to the structure of our crew from the original list.
- In the configuration file, set up the connection to the SQLite database.

**2. Initial Data Upload (Data Seeding):**
- In the main application file (e.g., `my_app.py`), take the existing `my_crew` list and write code that **once** inserts these four crew members into your new database table.
- **Key Maneuver:** Use a conditional check (`if not CrewMember.query.first():`) to ensure the data is loaded only on the first run when the databank is empty.

**3. Refactoring Read Protocols (READ operations):**
- We can remove the global `my_crew` variable from our application.
- Modify the view functions for the `/crew_members` and `/crew_members/<int:id>` routes so that they load data directly from your new database.

**4. Refactoring the Write Protocol (CREATE operation):**
- Modify the view function for the `/onboarding` route. The logic that added a new member to the list after form submission must now be replaced with logic that creates a new instance of the `CrewMember` model and saves it to the database.

**5. Final Systems Check:**
- Run the application and verify that you see 4 crew members loaded from the database on the `/crew_members` page.
- Using the form at `/onboarding`, add a new (fifth) member.
- **Restart the server** (stop and restart the application).
- Re-open the `/crew_members` page. If you see all 5 members, your mission was successful – you have created a persistent data store!

In [ ]:
# configuration.py

class Config:
    """
    Set Flask config variables
    """
    # General Config
    DEBUG = True 
    SECRET_KEY = 'my_secret_something'

    # Database Config (set up DB connection) - for SQLite
    SQLALCHEMY_DATABASE_URI = 'sqlite:///my_database.db' # path to the SQLite "instance" folder with ".db" file
    SQLALCHEMY_TRACK_MODIFICATIONS = False # disable warnings in terminal for session changes

In [ ]:
from flask import Flask
from config.configuration import Config 
from routes import register_routes
from database import db
from models import CrewMember


my_crew = [
        {
            "id": 1,
            "name": "Dave Fisher",
            "specialization": "AI Engineer",
            "status": "Active"
        },
        {
            "id": 2,
            "name": "Izael Alexander",
            "specialization": "Rocket Scientist",
            "status": "Active"
        },
        {
            "id": 3,
            "name": "Tethra Dyagran",
            "specialization": "Roboticist",
            "status": "Active"
        },
        {
            "id": 4,
            "name": "Mura Lan",
            "specialization": "Astronavigator",
            "status": "Active"
        }
    ]

my_asteroid = {"name": "230 Athamantis",
               "diameter": "118±2 km",
               "mass": "(2.3±1.1)*10**18 kg",
               "density": "2.7±1.3 g/cm3",
               "speed": "19.3 km/s",
               "img": "static/img/Athamantis.jpeg"
}


app = Flask(__name__)


# -------------- CONFIGURATION --------------
# ze souboru pomocí "třídy"
app.config.from_object(Config)


# -------------- INITIALIZATION --------------
db.init_app(app)
register_routes(app, db, my_asteroid)


# -------------- TABLEs - creation --------------
with app.app_context(): 
    db.create_all() # creates tables (if no tables in DB)
    if not CrewMember.query.first(): # populates database (if DB is empty)
        for crew_member in my_crew:
            new_crew_member = CrewMember(
                id=crew_member["id"], 
                name=crew_member["name"], 
                specialization=crew_member["specialization"], 
                status=crew_member["status"])
            db.session.add(new_crew_member)
        db.session.commit()


# -------------- STARTING THE APP --------------
if __name__ == "__main__":
    app.run()

In [ ]:
# routes.py

from flask import render_template, redirect, url_for
from forms import NewCrewMemberForm
from models import CrewMember


def register_routes(app, db, my_asteroid):
    @app.route("/")
    def homepage():
        return render_template("homepage.html")

    @app.route("/mission_briefing")
    def mission_briefing():
        return render_template("mission_briefing.html")

    @app.route("/target_asteroid")
    def target_asteroid():
        return render_template("target_asteroid.html", my_asteroid=my_asteroid)

    @app.route("/crew_members")
    def crew_members():
        my_crew = db.session.query(CrewMember).all()
        return render_template("crew_members.html", my_crew=my_crew)

    @app.route("/crew_members/<int:id>")
    def crew_members_details(id):
        crew_member = db.session.query(CrewMember).get(id)
        if crew_member:
            return render_template("crew_members_details.html", crew_member=crew_member)
        return "Crew member not found"
    
    @app.route("/onboarding", methods=["GET", "POST"])
    def onboarding():
        form = NewCrewMemberForm()
        if form.validate_on_submit():
            db.session.add(CrewMember(
                name=form.name.data,
                specialization=form.specialization.data,
                status="Pending.."
            ))
            db.session.commit()
            return redirect(url_for("crew_members"))
        return render_template("onboarding.html", form=form)

In [ ]:
# forms.py

from flask_wtf import FlaskForm
from wtforms import StringField, SubmitField, validators

class NewCrewMemberForm(FlaskForm):
    name = StringField(label="Name", validators=[validators.DataRequired()])
    specialization = StringField(label="Specialization", validators=[validators.DataRequired()])
    
    submit = SubmitField(label="Submit new crew member")

In [ ]:
# database.py

from flask_sqlalchemy import SQLAlchemy

db = SQLAlchemy()

In [ ]:
# models.py

from database import db # import 'db' object


class CrewMember(db.Model):
    __tablename__ = "crew_members"
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(100), nullable=False)
    specialization = db.Column(db.String(100), nullable=False)
    status = db.Column(db.String(100), nullable=False)

    def __repr__(self):
        return f"<CrewMember {self.name}>"

## /templates

In [ ]:
<!-- header.html -->

<nav>
    <ul>        
        <li><a href="{{ url_for('homepage') }}">Home</a></li> 
        <li><a href="{{ url_for('mission_briefing') }}">Our Mission</a></li>
        <li><a href="{{ url_for('target_asteroid') }}">The Asteroid</a></li>
        <li><a href="{{ url_for('crew_members') }}">Our Crew</a></li>
        <li><a href="{{ url_for('onboarding') }}">New Recruits Onboarding</a></li>
    </ul>
</nav>

In [ ]:
<!-- footer.html -->
 
<footer>
    <br>
    <p>Deep Space Exploration Corp.</p>
    <p>All rights reserved</p>
</footer>

In [ ]:
<!-- base.html -->
 
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>{% block page_title %} Title {% endblock page_title %}</title> 
    
</head>

<body>
    {% include 'header.html' %} 
    {% block page_content %} Content {% endblock page_content %} 
    {% include 'footer.html' %} 
</body>

</html>

In [ ]:
<!-- homepage.html -->

{% extends "base.html" %} 

{% block page_title %} HomePage {% endblock page_title %}

{% block page_content %}
    <h1>Deep Space Exploration Corp.</h1>
    <p>Welcome to our homepage !</p>
    <p>Our goal is to mine asteroids from the solar system.</p>
{% endblock page_content %}

In [ ]:
<!-- mission_briefing.html -->

{% extends "base.html" %} 

{% block page_title %} Mission Debriefing {% endblock page_title %}

{% block page_content %}
    <h1>Mission</h1>
    <p>This is our mission:</p>
    <p>get resources from chosen asteroids</p>
    <p>make it profitable</p>
    <p>expand world's economy</p>
{% endblock page_content %}

In [ ]:
<!-- target_asteroid.html -->

{% extends "base.html" %} 

{% block page_title %} Target Asteroid {% endblock page_title %}

{% block page_content %}
    <h1>Target asteroid: {{ my_asteroid["name"] }}</h1>
    <p>Mean diameter: {{ my_asteroid["diameter"] }}</p>
    <p>Mass: {{ my_asteroid["mass"] }}</p>
    <p>Mean density: {{ my_asteroid["density"] }}</p>
    <p>Average orbital speed: {{ my_asteroid["speed"] }}</p>
    <img src="{{ my_asteroid['img'] }}" width="800px"/> 
{% endblock page_content %}

In [ ]:
<!-- crew_members_details.html -->

{% extends "base.html" %} 

{% block page_title %} {{ crew_member["name"] }} {% endblock page_title %}

{% block page_content %}
    <h1>Crew member {{ crew_member["id"] }}</h1>
    <p>Name: {{ crew_member["name"] }}</p>
    <p>Specialization: {{ crew_member["specialization"] }}</p>
    <p>Status: {{ crew_member["status"] }}</p>
{% endblock page_content %}

In [ ]:
<!-- crew_members.html -->

{% extends "base.html" %}

{% block page_title %} Our Crew {% endblock page_title %}

{% block page_content %}
    {% for member in my_crew %}
        <p><a href="{{ url_for('crew_members_details', id=member['id']) }}">{{ member["name"] }}</a></p>
    {% endfor %}
{% endblock page_content %}

In [ ]:
<!-- onboarding.html -->

{% extends "base.html" %}

{% block page_title %} Onboarding {% endblock page_title %}

{% block page_content %}
<form method = "POST" action = "{{ url_for('onboarding') }}">
    <p>New crew member onboarding form</p>
    {{ form.hidden_tag() }} 

    <p>{{ form.name.label }} {{ form.name(size=30) }}</p>
    <p>{{ form.specialization.label }} {{ form.specialization(size=30) }}</p>

    <p>{{ form.submit() }}</p>
</form>
{% endblock page_content %}

---
#### © Jiří Svoboda (George Freedom)
- Web: https://GeorgeFreedom.com
- LinkedIn: https://www.linkedin.com/in/georgefreedom/
- Book me: https://cal.com/georgefreedom